In [1]:
import os
import itertools
import subprocess

class ChlamySimulationRunner:
    def __init__(self, 
                 output_dir="output_file", 
                 log_dir="logs", 
                 param_dir="params"):

        self.output_dir = output_dir
        self.log_dir = log_dir
        self.param_dir = param_dir

        os.makedirs(self.output_dir, exist_ok=True)
        os.makedirs(self.log_dir, exist_ok=True)
        os.makedirs(self.param_dir, exist_ok=True)

        self.Ns = [int(1e6)]
        self.taus = [1e-3]
        self.Rs = [5e-6]
        self.Hs = [100e-6]
        self.eta_0s = [1e-3]
        self.kbTs = [1e-15] ##
        self.vs = [0e-6] ##
        self.Ls = [88e-3]
        self.Ws = [180e-6]
        self.Delta_Ps = [1e3]
        self.Ts = [2.]
        self.n_taskss = [1]
        self.N_trajectoriess = [100]
        self.oscillationss = [1] # U can change this and pass it to run. too tired to do it
        
    def generate_param_file(self, filepath, **params):
        with open(filepath, "w") as f:
            for key, value in params.items():
                f.write(f"{key}={value}\n")

    def build_command(self, output_path, param_dict):
        base_cmd = [
            "python", "chlamy_run_simul.py",
            "-o", output_path
        ]
        for key, value in param_dict.items():
            flag = f"--{key}" if key not in ["N", "tau", "R", "H", "eta_0", "kbT", "v", "L", "W", "Delta_P", "T", "oscillations"] else f"-{key}"
            base_cmd.extend([flag, str(value)])
        return base_cmd

    def run_all(self):
        for N, tau, R, H, eta_0, kbT, v, L, W, Delta_P, T, n_tasks, N_trajectories, oscillations in itertools.product(
            self.Ns, self.taus, self.Rs, self.Hs, self.eta_0s, self.kbTs,
            self.vs, self.Ls, self.Ws, self.Delta_Ps, self.Ts, self.n_taskss, self.N_trajectoriess, self.oscillationss
        ):
            param_dict = {
                "N": N, "tau": tau, "R": R, "H": H, "eta_0": eta_0, "kbT": kbT,
                "v": v, "L": L, "W": W, "Delta_P": Delta_P, "T": T,
                "n_tasks": n_tasks, "N_trajectories": N_trajectories, "oscillations": oscillations
            }

            filename_base = f"{N}_{tau}_{R}_{H}_{eta_0}_{kbT}_{v}_{T}_{Delta_P}_{L}_{W}_{n_tasks}_{N_trajectories}_{oscillations}"
            output_path = self.output_dir #os.path.join(self.output_dir, filename_base)
            param_path = os.path.join(self.param_dir, filename_base + ".txt")
            log_path = os.path.join(self.log_dir, filename_base + ".out")

            self.generate_param_file(param_path, **param_dict)
            command = self.build_command(output_path, param_dict)

            with open(log_path, "w") as log_file:
                process = subprocess.run(command, stdout=log_file, stderr=subprocess.STDOUT)
                if process.returncode != 0:
                    print(f"Simulation failed for: {filename_base}. See log: {log_path}")


# Create an instance of the simulations to import parameters

In [2]:
import numpy as np


### Pressure array, log spaced, to match Brady plot velocities

In [3]:
pressure_array_logspaced = np.logspace(
        np.log(1.) / np.log(10), 
        np.log(100.) / np.log(10),
        8,
        base=10, )

pressure_array_logspaced = np.append(pressure_array_logspaced, 73.)
pressure_array_logspaced = np.append(pressure_array_logspaced, 0.)

# pressure_array_logspaced = [0., 100.,]

In [4]:
pressure_array_logspaced

array([  1.        ,   1.93069773,   3.72759372,   7.19685673,
        13.89495494,  26.82695795,  51.79474679, 100.        ,
        73.        ,   0.        ])

### Run all the simulations for the Brady plot

In [5]:
active_runner = ChlamySimulationRunner()
active_runner.output_dir = "./output_file/answer_referee_1_hydrodynamics_no_brenner_multiT"
active_runner.Delta_Ps = pressure_array_logspaced
active_runner.Ts = [2., 4., 8.]
active_runner.kbTs = [300 * 1.38 * 1e-23]
active_runner.oscillationss = [1]
active_runner.vs = [100e-6]
active_runner.run_all()

passive_runner = ChlamySimulationRunner()
passive_runner.output_dir = "./output_file/answer_referee_1_hydrodynamics_with_brenner"
passive_runner.Delta_Ps = pressure_array_logspaced
passive_runner.Ts = [2.]
passive_runner.kbTs = [1e-15]
passive_runner.oscillationss = [1]
passive_runner.vs = [0.]
passive_runner.run_all()